In [11]:
import pandas as pd
from datetime import datetime
import math

In [12]:
data = pd.read_csv("../00_data/merged_all_by_algo.csv", sep=";", decimal=",")
data.head(5)

,date,hour,datetime,day_ahead_price,intraday_price,price_diff,solar_capacity,solar_generation_actual,wind_capacity,wind_generation_actual,solar_generation_forecast,wind_generation_forecast
0,2016-01-01,1,2016-01-01 01:00:00,22.39,24.28,-1.89,39021,0,44325,8579,0,7421
1,2016-01-01,2,2016-01-01 02:00:00,20.59,25.26,-4.67,39021,0,44325,8542,0,7266
2,2016-01-01,3,2016-01-01 03:00:00,16.81,20.24,-3.43,39021,0,44325,8443,0,7180
3,2016-01-01,4,2016-01-01 04:00:00,17.41,21.94,-4.53,39021,0,44325,8295,0,7016
4,2016-01-01,5,2016-01-01 05:00:00,17.02,23.50,-6.48,39021,0,44325,7319,0,6787


In [15]:
data["total_generation_forecast"] = data["wind_generation_forecast"] + data["solar_generation_forecast"]
data["total_generation_actual"] = data["solar_generation_actual"]+data["wind_generation_actual"]
data["wind_forecast_error"] = data["wind_generation_forecast"] - data["wind_generation_actual"]
data["solar_forecast_error"] = data["solar_generation_forecast"] - data["solar_generation_actual"]
data["forecast_error"] = data["total_generation_forecast"] - data["total_generation_actual"]
data["forecast_error_percent"] = data["forecast_error"]/data["total_generation_forecast"]
data["pricediff_percent"] = data["price_diff"]/data["day_ahead_price"]
#season and weekday calculation
seasons = range(4)
data["season"] = data["date"].map(lambda x: seasons[math.ceil(datetime.strptime(x, '%Y-%m-%d').month/3)-1])
data["weekday"] = data["date"].map(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())
#print(data.head(5))

data.to_csv("../00_data/data_with_calc_fields.csv", sep=";", decimal=",", index=False)
print("done")

done
